In [2]:
import os  # Import the os module to interact with the operating system
import json  # Import the json module to work with JSON data
import requests  # Import the requests module to make HTTP requests
from azureml.core import Workspace  #  provides methods for managing resources within the workspace
from azureml.core.model import Model  #  allows you to register, manage, and deploy trained models within an Azure ML workspace
from azureml.core.environment import Environment  #  represents a Python environment that can be used for running machine learning experiments and deploying models
from azureml.core import Experiment  # provides methods for creating and managing experiments, as well as tracking experiment runs and metrics.
from azureml.core import ScriptRunConfig  #  allows you to specify the script file, environment, compute target, and other settings for the experiment run.
from azureml.core.conda_dependencies import CondaDependencies  # allows you to specify the Python packages and versions needed for model training and deployment.
from azureml.core.model import InferenceConfig  # defines the configuration for deploying a machine learning model as a web service for inference
from azureml.core.webservice import AciWebservice, Webservice  #  allows you to deploy models as web services to ACI for scalable and cost-effective inference.
from azureml.core.compute_target import ComputeTargetException  # an exception class used to indicate errors related to Azure Machine Learning compute targets. It is raised when a specified compute target cannot be found or accessed.
from azureml.core.compute import AmlCompute, ComputeTarget  # allows you to create, manage, and scale compute resources for model training and deployment.

In [7]:
config_file_path = "config.json"  # Specify the path to the configuration file (assumed to be in JSON format)

# Read JSON data from the configuration file into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)  # Load JSON data from the file into the 'data' dictionary

subscription_id = data["subscription_id"]  # Retrieve subscription ID from the 'data' dictionary
resource_group = data["resource_group"]  # Retrieve resource group name from the 'data' dictionary
workspace_name = data["workspace_name"]  # Retrieve workspace name from the 'data' dictionary
region = data["region"]  # Retrieve Azure region from the 'data' dictionary
compute_target = data["compute_target"]  # Retrieve compute target name from the 'data' dictionary

In [8]:
print(resource_group)
print(workspace_name)
print(region)
print(compute_target)

ML-RandD-AMS
WS-ML-RandD-AMS
eastus
amitsachdev


In [9]:
try:  # Try to load the Azure ML workspace configuration from the current environment
    ws = Workspace.from_config()  # Load the workspace configuration
    print('Workspace already exists.')
except:  # If the workspace configuration is not found, create a new workspace
    ws = Workspace.create(workspace_name,  # Specify the name for the new workspace
                          resource_group=resource_group,  # Specify the resource group for the new workspace
                          create_resource_group=True,  # Create the resource group if it doesn't exist
                          subscription_id=subscription_id,  # Specify the subscription ID
                          region=region)  # Specify the Azure region for the workspace
    ws.write_config('.azureml')  # Write the workspace configuration to a file for future use

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Workspace already exists.


In [7]:
Workspace.from_config()  # Load the Azure ML workspace configuration from the current environment

Workspace.create(name='WS-ML-RandD-AMS', subscription_id='4058d79e-a063-4ebf-bf0e-9f1ab5529019', resource_group='ML-RandD-AMS')

In [10]:
try:  # Try to retrieve an existing compute target
    aml_compute = AmlCompute(ws, compute_target)  # Load the existing Azure ML compute target
    print("This Compute Target already exists.")
except ComputeTargetException:  # If the compute target doesn't exist, create a new one
    print("Creating new compute target:", compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",  # Specify the VM size for the compute target
                                                                min_nodes=1,  # Specify the minimum number of nodes
                                                                max_nodes=4,  # Specify the maximum number of nodes
                                                                idle_seconds_before_scaledown=3000)  # Specify the idle time before scaling down
    aml_compute = ComputeTarget.create(ws, compute_target, provisioning_config)  # Create a new Azure ML compute target
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)  # Wait for the provisioning to complete

    
print("Azure Machine Learning Compute attached now")

This Compute Target already exists.
Azure Machine Learning Compute attached now


In [11]:
model_path = 'Best_model.pkl'  # Specify the path to the trained model file

In [12]:
model_name = 'Housing_price_prediction'  # Specify the name for the registered model

In [13]:
registered_model = Model.register(model_path=model_path,  # Register the model in the Azure ML workspace
                                  model_name=model_name,  # Specify the name for the registered model
                                  workspace=ws)  # Set the Azure ML workspace for registration

Registering model Housing_price_prediction


In [14]:
conda_env = Environment('AzureDeploy-Housing')  # Create a new environment with the specified name
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])  # Specify the Conda dependencies for the environment

In [15]:
inference_config = InferenceConfig(entry_script='Score.py',  # Specify the entry script for inference
                                   environment=conda_env)  # Set the environment configuration for inference

In [16]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=5)  # Specify the deployment configuration for ACI

In [18]:
service = Model.deploy(workspace=ws,  # Deploy the model as a web service in Azure ML
                       name='housing-price-prediction',  # Specify the name for the deployed service
                       models=[registered_model],  # Use the registered model for deployment
                       inference_config=inference_config,  # Set the inference configuration for the service
                       deployment_config=aci_config)  # Set the deployment configuration for the service

service.wait_for_deployment(show_output=True)  # Wait for the deployment to complete and show the output

C:\Users\Chirayu Baliyan\AppData\Local\Temp\ipykernel_13184\2644788263.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,  # Deploy the model as a web service in Azure ML


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-04-18 00:02:50+05:30 Creating Container Registry if not exists.
2024-04-18 00:02:50+05:30 Registering the environment.
2024-04-18 00:02:52+05:30 Use the existing image.
2024-04-18 00:02:52+05:30 Generating deployment configuration.
2024-04-18 00:02:53+05:30 Submitting deployment to compute.
2024-04-18 00:02:58+05:30 Checking the status of deployment housing-price-prediction..
2024-04-18 00:04:24+05:30 Checking the status of inference endpoint housing-price-prediction.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
scoring_uri = service.scoring_uri  # Get the scoring URI for the deployed service

In [20]:
scoring_uri

'http://e332cf5e-308c-46f6-8f56-d98c139094ed.eastus.azurecontainer.io/score'